In [14]:
#import streamlit as st
import pandas as pd

base_curves = pd.DataFrame(pd.read_csv('all_kfc_curves.csv'))

curves_only = base_curves.drop('netspend',axis=1).reset_index()

def optimise_curve(df, budget, weeks):
    # create a new df to hold the results
    df_mid = pd.DataFrame()

    #calculate the number of 'units' that we have to assign
    units = budget/weeks/1000
    
    for col in df.columns:
        df_mid[col + '_incr'] = df[col].diff().fillna(0)
    
    df_mid = df_mid.filter(regex='_incr').melt(var_name = 'campaign').nlargest(round(units), 'value')

    budget_df = pd.DataFrame(df_mid['campaign'].value_counts()).reset_index()

    budget_df['campaign'] = budget_df['campaign'].str.replace('_incr', '')

    budget_df['budget'] = (budget_df['count']/(budget_df['count'].sum()))*budget

    budget_df = budget_df.drop('count',axis=1)

    budget_df['budget'] = budget_df['budget'].round()
    
    return budget_df

df_filtered = base_curves[['Lunch_Social', 'Lunch_TV']]

final_df = optimise_curve(df_filtered, weeks = 10, budget = 10000000).reset_index()

print(final_df)

   index      campaign     budget
0      0      Lunch_TV  8090000.0
1      1  Lunch_Social  1910000.0


In [9]:
#settings
df = df_filtered
budget = 10000000
weeks = 10
col = 'Lunch_Social'

Lunch_TV_wklybdgt = 809000
Lunch_Social_wklybdgt = 191000


In [18]:
final_df['wklybudget'] = final_df['budget']/weeks

print(final_df)

for row in final_df.rows:
        df_mid[col + '_incr'] = df[col].diff().fillna(0)

   index      campaign     budget  wklybudget
0      0      Lunch_TV  8090000.0    809000.0
1      1  Lunch_Social  1910000.0    191000.0
